In [37]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Data

In [38]:
mnist_collection, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

W0821 21:12:26.160934  6268 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [39]:
mnist_train, mnist_test = mnist_collection['train'], mnist_collection['test']
num_validation_samples = 0.1* mnist_info.splits['train'].num_examples 

In [40]:
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [41]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [42]:
def scaling(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label
scaled_train_and_validation_data = mnist_train.map(scaling)
scaled_test_data = mnist_test.map(scaling)

# Shuffling

In [43]:
BUFFER_SIZE = 10000

In [44]:
shuffled_train_and_val_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_val_data.take(num_validation_samples)
train_data = shuffled_train_and_val_data.skip(num_validation_samples)
BATCH_SIZE = 100

In [45]:
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)
val_inputs, val_targets = next(iter(validation_data))




# Model

Outline the model

In [46]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax'),
])

In [47]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Training

In [48]:
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(val_inputs,val_targets), verbose=2)

Epoch 1/5
540/540 - 8s - loss: 0.4341 - accuracy: 0.8777 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 7s - loss: 0.1913 - accuracy: 0.9447 - val_loss: 0.1919 - val_accuracy: 0.9465
Epoch 3/5
540/540 - 13s - loss: 0.1433 - accuracy: 0.9581 - val_loss: 0.1591 - val_accuracy: 0.9545
Epoch 4/5
540/540 - 8s - loss: 0.1181 - accuracy: 0.9649 - val_loss: 0.1348 - val_accuracy: 0.9627
Epoch 5/5
540/540 - 13s - loss: 0.1001 - accuracy: 0.9702 - val_loss: 0.1168 - val_accuracy: 0.9663


# Testing model

In [49]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 1s 1s/step - loss: 0.1121 - accuracy: 0.96 - 1s 1s/step - loss: 0.1121 - accuracy: 0.9663

In [53]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss: 0.11. Test accuracy: 96.63%


### Test accuracy is close to validation accuracy so there is not overfitting